%matplotlib inline

In [1]:

import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import cv2
from PIL import Image
from random import shuffle
import math

import pandas as pd

import pickle

from sklearn.svm import SVC,LinearSVC
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier,ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing

#from scipy.misc import imread, imresize



In [2]:
train_df = pd.read_csv('../../data/Manually_Annotated_file_lists/training_face_mesh_crop.csv')
train_df.subDirectory_filePath = '../../data/Manually_Annotated_Images_FaceMesh_Cropped/' + train_df.subDirectory_filePath

#train_df_2 = pd.read_csv('../../data/Automatically_annotated_file_list/automatically_annotated_face_mesh_crop.csv')
#train_df_2.subDirectory_filePath = '../../data/Automatically_Annotated_Images_FaceMesh_Cropped/' + train_df_2.subDirectory_filePath
#train_df = train_df.append(train_df_2)
#del train_df_2

train_df = train_df[train_df['have_facemesh']]

class DataFrameBatchIterator:
    def __init__(self, dataframe, batch_size):
        self.df = dataframe
        self.index = 0
        self.batch_size = batch_size

    def __iter__(self):
        return self
        
    def __next__(self):
        if self.index < len(self.df):
            self.index += self.batch_size
            return self.df[self.index - self.batch_size: self.index].copy()
        else:
            raise StopIteration

In [ ]:
from eason_utils import DataFrameBatchIterator

In [4]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
config = tf.compat.v1.ConfigProto(gpu_options = tf.compat.v1.GPUOptions(allow_growth = True))
sess = tf.compat.v1.Session(config = config)

In [5]:
mobilenet_pretrained = tf.keras.models.load_model('../models/affectnet_emotions/mobilenet_7.h5')

In [6]:
mobilenet_output = mobilenet_pretrained.get_layer("global_pooling").output

In [7]:
x = (mobilenet_output)
for size in [256]:
    x = tf.keras.layers.Dense(size, activation = 'relu')(x)

In [8]:
outputs_valance = tf.keras.layers.Dense(1, activation = 'sigmoid', name = 'outputs_valance')(x)

In [9]:
outputs_arousal = tf.keras.layers.Dense(1, activation = 'sigmoid', name = 'outputs_arousal')(x)

In [10]:
emotions_count = len(np.unique(train_df.expression))
outputs_emotion = tf.keras.layers.Dense(emotions_count, activation = 'softmax', name = 'outputs_emotion')(x)

In [11]:
model = tf.keras.Model(inputs=mobilenet_pretrained.input,
                       outputs=(outputs_valance, outputs_arousal, outputs_emotion) , name="mobilenet_train")

In [12]:
model.summary()

Model: "mobilenet_train"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 225, 225, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1 (Conv2D)                 (None, 112, 112, 32  864         ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                    

In [13]:
from uuid import uuid4

In [14]:
from datetime import datetime,tzinfo,timedelta

In [15]:
class Zone(tzinfo):
    def __init__(self,offset,isdst,name):
        self.offset = offset
        self.isdst = isdst
        self.name = name
    def utcoffset(self, dt):
        return timedelta(hours=self.offset) + self.dst(dt)
    def dst(self, dt):
            return timedelta(hours=1) if self.isdst else timedelta(0)
    def tzname(self,dt):
         return self.name

In [16]:
d = datetime.now(tz=Zone(8,False,'GMT'))
now_time_string = d.strftime("%m_%d_%H:%M:%S")
log_file_name = f"{now_time_string}.log"
print(f"{log_file_name=}")
def lprint(text, *args, **kwargs):
    texts = [text] + list(args)
    texts = map(lambda x: str(x), texts)
    text = ' '.join(texts)
    print(text)
    with open(log_file_name, 'a') as f:
        f.write(datetime.now(tz=Zone(8,False,'GMT')).strftime("%m_%d_%H:%M:%S ") + text + "\n")

log_file_name='01_19_22:57:20.log'


In [17]:
lprint("Hello")

Hello


In [18]:
#model.layers[1].trainable = False

In [19]:
# Instantiate an optimizer.
optimizer = tf.keras.optimizers.Adam()
# Instantiate a loss function.
MSE_loss = tf.keras.losses.MeanSquaredError()
SCC_loss = tf.keras.losses.SparseCategoricalCrossentropy()

In [20]:
batch_size = 32
epochs = 4

In [ ]:
from telegram_notifier import send_message
import time
start_time = time.time()

total_loss = 0
total_step = 0
for epoch in range(epochs):
    for step, row in enumerate(DataFrameBatchIterator(train_df, batch_size=batch_size)):

        imgs = row.subDirectory_filePath.apply(lambda x: cv2.resize(
            cv2.cvtColor(cv2.imread(x), cv2.COLOR_BGR2RGB), (224, 224)))
        img_array = np.array(list(imgs))

        y_valence = np.array(row.valence)
        y_arousal = np.array(row.arousal)
        y_emotion = np.array(row.expression)

        with tf.GradientTape() as tape:
            logits = model(img_array, training=True)

            pred_valence = logits[0]
            pred_arousal = logits[1]
            pred_emotion = logits[2]

            valence_loss = MSE_loss(y_valence, pred_valence)
            arousal_loss = MSE_loss(y_arousal, pred_arousal)
            emotion_loss = SCC_loss(y_emotion, pred_emotion)

            loss = valence_loss + arousal_loss + emotion_loss

            grads = tape.gradient(loss, model.trainable_weights)

            optimizer.apply_gradients(zip(grads, model.trainable_weights))
        total_loss += float(loss)
        total_step += 1

        valence_loss = float(valence_loss)
        arousal_loss = float(arousal_loss)
        emotion_loss = float(emotion_loss)
        if step % 10 == 0:
            lprint(
                f"Training loss at epoch {epoch} step {step}: {total_loss/total_step}\n",
                f"This round's {valence_loss=}, {arousal_loss=}, {emotion_loss=}"
            )
            lprint("Seen so far: %s samples" % ((step + 1) * batch_size))
            lprint(f"--- {time.time() - start_time} seconds for iter {total_step} step, each step have {batch_size}, so the model train with {total_step*batch_size} img this iter ---\n")
            total_loss = 0
            total_step = 0

            start_time = time.time()
    model.save(f"models/{now_time_string}_epoch{epoch}_batch{batch_size}")
    send_message(f"epoch {epoch} is finish")

Training loss at epoch 0 step 0: 8.087566375732422
 This round's valence_loss=2.7215681076049805, arousal_loss=2.8208649158477783, emotion_loss=2.545133113861084
Seen so far: 32 samples
--- 2.6277694702148438 seconds for iter 1 step, each step have 32, so the model train with 32 img this iter ---

Training loss at epoch 0 step 10: 3.6479535579681395
 This round's valence_loss=0.7429653406143188, arousal_loss=0.5692674517631531, emotion_loss=0.9357542991638184
Seen so far: 352 samples
--- 2.347302198410034 seconds for iter 10 step, each step have 32, so the model train with 320 img this iter ---

Training loss at epoch 0 step 20: 3.7847675561904905
 This round's valence_loss=1.137237310409546, arousal_loss=0.9827213287353516, emotion_loss=1.3837642669677734
Seen so far: 672 samples
--- 2.1026318073272705 seconds for iter 10 step, each step have 32, so the model train with 320 img this iter ---

Training loss at epoch 0 step 30: 3.559966230392456
 This round's valence_loss=1.111235857009

In [46]:
eval_df = pd.read_csv('../../data/Manually_Annotated_file_lists/validation_face_mesh_crop.csv')
eval_df.subDirectory_filePath = '../../data/Manually_Annotated_Images_FaceMesh_Cropped/' + eval_df.subDirectory_filePath


eval_df = eval_df[eval_df['have_facemesh']]

In [49]:
import tqdm

In [60]:
total_process = 0
start_time = time.time()

total_loss = 0
total_step = 0
total_valence_loss = 0
total_arousal_loss = 0
total_emotion_loss = 0
for step, row in tqdm.tqdm(enumerate(DataFrameBatchIterator(eval_df, batch_size=batch_size)), total = len(eval_df) / batch_size):
    if len(row) == 0:
        continue
    imgs = row.subDirectory_filePath.apply(lambda x: cv2.resize(
        cv2.cvtColor(cv2.imread(x), cv2.COLOR_BGR2RGB), (224, 224)))

    img_array = np.array(list(imgs))

    y_valence = np.array(row.valence)
    y_arousal = np.array(row.arousal)
    y_emotion = np.array(row.expression)


    logits = model(img_array, training=True)

    pred_valence = logits[0]
    pred_arousal = logits[1]
    pred_emotion = logits[2]

    valence_loss = MSE_loss(y_valence, pred_valence)
    arousal_loss = MSE_loss(y_arousal, pred_arousal)
    emotion_loss = SCC_loss(y_emotion, pred_emotion)

    loss = valence_loss + arousal_loss + emotion_loss

    total_loss += float(loss)
    total_step += 1
    total_process += len(img_array)

    total_valence_loss += float(valence_loss)
    total_arousal_loss += float(arousal_loss)
    total_emotion_loss += float(emotion_loss)
lprint(f"--- {time.time() - start_time} seconds for evaluate {total_process} image ---\n")

12710it [00:28, 439.72it/s]                                                     

--- 28.90762186050415 seconds for evaluate 5452 image ---



In [62]:
total_loss/total_process

0.1227161291759852

In [64]:
total_valence_loss/total_process

0.0287551013636659

In [65]:
total_arousal_loss/total_process

0.02919404697894875

In [66]:
total_emotion_loss/total_process

0.06476698066938125

In [ ]:
model.evaluate()

In [44]:
total_process

360480

In [45]:
eval_df

,subDirectory_filePath,face_x,face_y,face_width,face_height,facial_landmarks,expression,valence,arousal,have_facemesh
0,../../data/Manually_Annotated_Images_FaceMesh_...,134.0,134.0,899,899,181.64;530.91;188.32;627.82;195.1;723.37;205.2...,1,0.785714,-0.055556,True
1,../../data/Manually_Annotated_Images_FaceMesh_...,20.0,20.0,137,137,28.82;77.52;29.12;93.25;31.04;108.51;33.03;123...,0,-0.017253,0.004313,True
2,../../data/Manually_Annotated_Images_FaceMesh_...,11.0,11.0,176,176,30.52;87.33;32.55;106.43;36.94;125.81;43.06;14...,0,0.174603,0.007937,True
3,../../data/Manually_Annotated_Images_FaceMesh_...,40.0,40.0,269,269,44.43;158.17;47.08;189.2;50.54;221.88;58.3;253...,1,0.153401,0.038890,True
4,../../data/Manually_Annotated_Images_FaceMesh_...,22.0,22.0,153,153,50.59;78.72;48.6;93.23;48.72;109.06;48.8;123.0...,8,0.783972,-0.551684,True
...,...,...,...,...,...,...,...,...,...,...
414795,../../data/Manually_Annotated_Images_FaceMesh_...,52.0,52.0,348,348,83.9;225.92;88.93;260.62;97.28;296.78;108.34;3...,2,-0.662960,-0.249501,True
414796,../../data/Manually_Annotated_Images_FaceMesh_...,49.0,49.0,330,330,106.87;161.43;105.49;191.66;109.81;224.48;113....,2,-0.833333,-0.380952,True
414797,../../data/Manually_Annotated_Images_FaceMesh_...,28.0,28.0,191,191,52.53;97.98;50.5;116.98;50.65;136.12;53.3;155....,1,0.542562,-0.015803,True
414798,../../data/Manually_Annotated_Images_FaceMesh_...,65.0,65.0,724,724,95.85;340.5;99.96;432.63;117.87;521.21;138.52;...,10,-2.000000,-2.000000,True
